In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 21.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 86.7 MB/s eta 0:00:00:00:0100:01


In [2]:
# load: 
import os 
import json
import numpy as np
import pandas as pd
import re
from tqdm import tqdm
import time
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

In [5]:
# read the augmented csv file: 
merged_df = pd.read_csv('rating_disagreement_combined_augmented_data.csv')
merged_df.combine_tag.value_counts()

0    10556
1     1174
Name: combine_tag, dtype: int64

In [6]:
# load the pre-trained models: 
from transformers import RobertaTokenizer, TFRobertaModel
with tf.device('/device:GPU:0'):
    tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
    model = TFRobertaModel.from_pretrained("roberta-base")

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


In [7]:
# train test split - split trainning into train and validation, take 10% as validation
train, test_df = train_test_split(merged_df, test_size= 0.3, random_state=42, stratify=merged_df['combine_tag'])

In [8]:
# train test split - split trainning into train and validation, take 10% as validation
train_df_1, val_df = train_test_split(train, test_size= 0.2, random_state=42, stratify=train['combine_tag'])

In [9]:
val_df.combine_tag.value_counts()

0    1479
1     164
Name: combine_tag, dtype: int64

In [10]:

class create_tensor_dataset(): 
    
    '''Purpose: Create a tensorflow dataset for model training
    df_to_convert: The dataframe of interest, that contains the 
    text columns
    text: The column within the dataframe containing the text 
    batch_size: int, batch size
    tokenizer: The tokenizer to be utilised
    labels: The class/label column within the dataframe
    '''
    
    def __init__(self, df_to_convert, text,target_col, batch_size, labels=None ): 
        self.df_to_convert = df_to_convert
        self.text = text # column containing the text (here chunked_data)
        self.target_col = target_col # y column of interest here label
        self.batch_size = batch_size
        self.tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
        self.labels = labels # list of all the labels 
        
    def generate_training_data(self, ids, masks):
        for i, text in tqdm(enumerate(self.df_to_convert[self.text])):
            with tf.device('/device:GPU:0'):
                
                tokenized_text =  self.tokenizer.encode_plus(
                    text,# The sequence or batch of sequences to be encoded.
                    max_length=256, #Controls the maximum length to use by one of the truncation/padding parameters.
                    truncation=True, # Truncate to a maximum length specified with the argument max_length
                    padding='max_length', #pad to a maximum length specified with the argument max_length 
                    add_special_tokens=True,# encode the sequences with the special tokens relative to their model.
                    return_tensors='tf'   #return TensorFlow tf.constant objects.
                )
                ids[i, :] = tokenized_text.input_ids ##List of token ids to be fed to a model
                masks[i, :] = tokenized_text.attention_mask #List of indices specifying which tokens should be attended to by the model
        return ids, masks
    
    def ohe_label(self): 
         # convert label values to a list 
        return self.df_to_convert[self.target_col].values.tolist()

    def DatasetMapFunction(self, input_ids, attn_masks, labels):
        # create and return key-value pair of input ids, attention mask and the corresnponding labels
        return {
            'input_ids': input_ids,
            'attention_mask': attn_masks
        }, labels
    
    def tensor_data(self): 
        # input_ids array
        X_input_ids = np.zeros((len(self.df_to_convert), 256))
        # attention mask array
        X_attn_masks = np.zeros((len(self.df_to_convert), 256))
        # retrieve the input ids and attention masks
        X_input_ids, X_attn_masks = self.generate_training_data(X_input_ids, X_attn_masks)
        # creation of tensorflow dataset: 
        dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, self.ohe_label()))
        dataset = dataset.map(self.DatasetMapFunction)
        dataset = dataset.shuffle(10000).batch(self.batch_size, drop_remainder=True)
        return dataset

In [11]:
# creation of tensorflow dataset for training
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)
train_tensor =create_tensor_dataset(train_df_1, 'full_review', 'combine_tag',8 )
train_dataset =train_tensor.tensor_data() 

6568it [00:07, 865.92it/s] 


In [12]:
# validation tensor dataset: 
with tf.device('/device:GPU:0'):
    val_tensor =create_tensor_dataset(val_df, 'full_review', 'combine_tag', 8)
    val_dataset =val_tensor.tensor_data() 

1643it [00:02, 768.97it/s]


In [13]:

# defining 2 input layers for input_ids and attn_masks
with tf.device('/device:GPU:0'):
    input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
    attn_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')
    bert_embds = model.roberta(input_ids, attention_mask=attn_masks)[1] # pooled output
    # intermediate_layer_1 = tf.keras.layers.Dropout(0.5, name="dropout")(bert_embds)# 0 -> activation layer (3D), 1 -> pooled output layer (2D)
    intermediate_layer = tf.keras.layers.Dense(1024, activation='relu', name='intermediate_layer')(bert_embds)
#     intermediate_layer_2 = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer_2')(intermediate_layer)
    # intermediate_layer_2 = tf.keras.layers.Dropout(0.4, name="dropout2")(intermediate_layer)
    output_layer = tf.keras.layers.Dense(1, activation='sigmoid', name='output_layer')(intermediate_layer) # sigmoid -> calcs probs of binary classes
    roberta_imdb = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
    roberta_imdb.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 roberta (TFRobertaMainLayer)   TFBaseModelOutputWi  124645632   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                           

In [14]:
# utilization of adam optimization 
optim = tf.keras.optimizers.Adam(learning_rate=1e-5)
# utilised binary cross entropy loss 
loss_func = tf.keras.losses.BinaryCrossentropy()
roberta_imdb.compile(optimizer=optim, loss=loss_func, metrics=['accuracy'])

# early stopping - administer val_loss: 
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping()
custom_early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience= 3,
    mode='min'
)

with tf.device('/device:GPU:0'):

    hist = roberta_imdb.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=100, 
        callbacks =[custom_early_stopping]
    )

Epoch 1/100
821/821 [==============================] - 405s 455ms/step - loss: 0.2133 - accuracy: 0.9304 - val_loss: 0.0928 - val_accuracy: 0.9689
Epoch 2/100
821/821 [==============================] - 367s 447ms/step - loss: 0.0892 - accuracy: 0.9740 - val_loss: 0.0891 - val_accuracy: 0.9750
Epoch 3/100
821/821 [==============================] - 367s 447ms/step - loss: 0.0613 - accuracy: 0.9816 - val_loss: 0.0988 - val_accuracy: 0.9762
Epoch 4/100
821/821 [==============================] - 365s 445ms/step - loss: 0.0383 - accuracy: 0.9889 - val_loss: 0.0940 - val_accuracy: 0.9720
Epoch 5/100
821/821 [==============================] - 365s 445ms/step - loss: 0.0255 - accuracy: 0.9918 - val_loss: 0.0988 - val_accuracy: 0.9774


In [15]:
def prepare_data(input_text, tokenizer):
    with tf.device('/device:GPU:0'):
        
        token = tokenizer.encode_plus(
            input_text,
            max_length=256, 
            truncation=True, 
            padding='max_length', 
            add_special_tokens=True,
            return_tensors='tf'
        )
        return {
            'input_ids': tf.cast(token.input_ids, tf.float64),
            'attention_mask': tf.cast(token.attention_mask, tf.float64)
        }

def make_predictions(model, input_text,threshold, label_list=None ): 
    with tf.device('/device:GPU:0'):
        processed_data = prepare_data(input_text, tokenizer)
        probs = model.predict(processed_data)
#         print(probs)
        if probs[0]> threshold: 
            return label_list[0]
        else: 
            return label_list[1]
#     return(probs[0])

In [16]:
# tt = test_df.head(20)
label_list= [1, 0]
test_df['result']= test_df.full_review.apply(lambda x: make_predictions(roberta_imdb, x, 0.5, label_list))

1/1 [==============================] - 0s 60ms/step


1/1 [==============================] - 0s 60ms/step


1/1 [==============================] - 0s 61ms/step


1/1 [==============================] - 0s 60ms/step


1/1 [==============================] - 0s 61ms/step


...

1/1 [==============================] - 0s 62ms/step


1/1 [==============================] - 0s 61ms/step


1/1 [==============================] - 0s 60ms/step


1/1 [==============================] - 0s 65ms/step


1/1 [==============================] - 0s 63ms/step


In [17]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
print (classification_report(test_df['combine_tag'], test_df['result']))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98      3167
           1       0.82      0.85      0.83       352

    accuracy                           0.97      3519
   macro avg       0.90      0.91      0.91      3519
weighted avg       0.97      0.97      0.97      3519



In [18]:
print('done')

done


In [19]:
roberta_imdb.save('augmented_combined_model_rating_disagreement')

INFO:tensorflow:Assets written to: augmented_combined_model_rating_disagreement/assets


INFO:tensorflow:Assets written to: augmented_combined_model_rating_disagreement/assets


In [51]:

test_df[test_df['result']==1]

,review_id,full_review,rating_managment_explicit,rating_managment_explicit_ohe,disagreement_with_ratings_ohe,combine_tag,result
3711,R21E0B01DWGBBD,This is not a review of this product. I just f...,1,1,0,1,1
4191,R2KCZJ06LGQHSK,i don't know how they're getting so many poz r...,0,0,1,1,1
3727,R10OM6C2QVMF5Y,"When I opened the black cartridge, it started ...",0,0,1,1,1
4100,R2H2HTE201IKWX,"Don't buy if you're taller than 5' 2"". The ha...",0,0,0,0,1
3071,R3LJC3I37JYOH2,Doesn't work with any Apple devices wirelessly...,0,0,0,0,1
...,...,...,...,...,...,...,...
332,R1DVMXGRXAC20T,I should have read more of the reviews before ...,0,0,1,1,1
1455,R2XK99VYP2BAXC,BUYER BEWARE!!! I ordered theese and none of t...,1,1,0,1,1
2412,R22GTFLV2Y3LNW,Buyer Beware! Have no idea what world the oth...,0,0,1,1,1
944,R1RCL5QPWJMILW,I learned cursive with a fountain pen so I'm n...,0,0,1,1,1


In [26]:
test_df[test_df['result']==1].iloc[90].full_review

'This is a flimsy product with  strips of magnetic tape not magnets attached. It does not adhere to my whiteboard and slides down other metallic surfaces. I was greatly disappointed  that the other reviews were so misleading. If anyone know of a good  protractor for a whiteboard let me know, this is not it.'

In [23]:
test_df[test_df['result']==1].iloc[4].full_review

'Just one listen...that\'s all it took. Now before you give me the old "this album has to be absorbed slowly" bullcrap, let me say that normally I will give new albums their due time to sink in and Porcupine Tree is certainly a band who\'s work normally calls for that. Yet there is nothing in this album that even comes close to requiring attentive and repeated listening, and that\'s the main problem with "Deadwing"."Deadwing" is prog...I get it. "Deadwing" has some metal parts...I get it. "Deadwing" has cute little electronic blips and bloops in the background...I get it. "Deadwing" has effects-laden vocals...I get it. The reason I got it all on the first listen is because I\'ve heard this all before, mostly from inferior bands.Good experimental music will always demand lots of time for deciphering...only when the riddles have been solved and the codes broken can a decision be made on whether or not the listener can connect with the message, relate to it and enjoy it. So why are all th

In [32]:
test_df[test_df['result']==1].iloc[9].full_review

'There is only one ream of paper so be careful when you make this purchase you can get cheaper paper and for much cheaper.. not sure how people reviewed this product.'